#### Предобработка данных

In [ ]:
import re
import pymystem3
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
from utils import *

In [ ]:
def clean(s):
    s = re.sub('\n', ' ', s) # 1
    s = re.sub('<\?xml.+\?>', ' ', s) # 2
    s = re.sub('<\/?[A-Za-z]*>', ' ', s) # 3
    s = re.sub('[0-9]+(\.|,)?[0-9]*', ' num ', s) # 4
    s = re.sub('([a-zа-яёй])([A-ZА-ЯЁЙ])', '\g<1> \g<2>', s) # 5
    s = re.sub('[^A-Za-zА-ЯЁЙа-яёй0-9\s\-]', ' ', s) # 6
    return re.sub('\s{2,}', ' ', s).strip().lower() # 7

1. *удаляем символы переноса строки `\n`*
2. *удаляем `< ?xml version="1.0" encoding="utf-8"? >`*
3. *удаляем `< conversion >`, `< person >` и прочие теги*
4. *заменяем числа `13`, `13.3`, `13,3` на тег `num`*
5. *"расклеиваем" слова (где это очевидно) `болееТысячи` -> `более Тысячи`*
6. *оставляем только символы латинского алфавита и кириллицы, а также цифры, пробелы и дефисы*
7. *удаляем лишние пробелы, удаляем пробелы слева и справа, приводим к нижнему регистру*

In [ ]:
def tokenize(s):
    return re.findall('[.,;:!?()"]|\w+-\w+|\w+\'\w+|\w+|-|\'', s)

* *разделяем текст на токены:*
* *символы из последовательности `.,;:!?()"`*
* *слова с дефисом `кто-то` и апострофом `didn't` (мало ли :)*
* *одиночные слова, а также символы дефиса и апострофа отдельно*

In [ ]:
def lemmatize(words, stem):
    return [stem.lemmatize(w)[0].strip() for w in words]

* *производим лемматизацию: `тысячи` -> `тысяча`*
* *удаляем символ `\n` (особенность pymystem3)*

In [ ]:
def remove_stopwords(words, stopwords):
    return [w for w in words if w not in set(stopwords)]

* *удаляем стоп-слова (слишком часто встречающиеся и потому не несущие полезную информацию)*
* *имеет смысл поискать такой список под данный коркретный домен (к которому относятся наши тексты)*
* *кроме того, думаю, имеет смысл найти / создать список стоп-слов, которые не нужно лемматизировать*

In [ ]:
def preprocess(text, stem, stopwords):
    words = lemmatize(tokenize(clean(text)), stem)
    return ' '.join(remove_stopwords(words, stopwords))

* *объединяем все вместе: на входе исходный текст, на выходе - предобработанный*

In [ ]:
tqdm.pandas()
stem = pymystem3.Mystem()
s1 = load_lines('data/iso_stopwords_ru.txt')
s2 = load_lines('data/nltk_stopwords_en.txt')
fp = lambda s: preprocess(s, stem, s1 + s2)
df = pd.read_csv('data/raw.csv', nrows=None)

* *инициализируем pymystem3*
* *загружаем список стоп-слов*
* *загружаем исходный датафрейм*

In [ ]:
labels = encode_labels(df['topic'])
df['topic'] = df['topic'].map(labels)
df['text'] = df['text'].progress_apply(fp)
df['title'] = df['title'].progress_apply(fp)
df['title_len'] = df['title'].str.len()
df['text_len'] = df['text'].str.len()
dump_json(labels, 'data/labels.json')
df.to_feather('data/prep.ftr')

* *предобрабатываем тексты и заголовоки*
* *формируем новые признаки: text_len и title_len*
* *кодируем целевые метки и сохраняем их в json файл*

In [3]:
corpus = df['title'] + ' ' + df['text']
dump_lines(corpus, 'data/corpus.txt')

* *сохраняем корпус текстов в отдельный файл для удобства последующего предобучения*